<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_DataFrames_WK01CL02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Pandas

## Setup and preliminaries

In order to read and process files, we are going to use a very powerful, and widely used Python library, called pandas. So, our next step is to import the pandas library in Python, and also import the library matplotlib for generating plots:

In [ ]:
import pandas as pd
import numpy as np

# Data Types and Conversions

# Loading Data

## From CSV Files

We will use a dataset with [restaurant inspection results in NYC](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j).

We fetch it by executing the following command:

In [ ]:
# Fetches the most recent dataset
!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o restaurant.csv

We want to be able to read and process this file within Python. The pandas library has a very convenient method `read_csv` which reads the file, and returns back a variable that contains its contents.

In [ ]:
import pandas as pd

restaurants = pd.read_csv(
    "restaurant.csv",
    encoding="utf_8",
    dtype="unicode",
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False,
)

When you read a CSV, you get back a kind of object called a DataFrame, which is made up of rows and columns. You get columns out of a DataFrame the same way you get elements out of a dictionary. Let's take a look at how the object looks like:

In [ ]:
restaurants.head(5)

The read_csv method has many options, and you can read further in the [online documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.parsers.read_csv.html).

We can also check the data types for each column

In [ ]:
restaurants.dtypes

We can use the method "describe()" to get a quick overview of the data in the dataframe.

In [ ]:
restaurants.describe()

In [ ]:
# Same as above, but the .T command transposes the table
restaurants.describe().T

The `object` type is a string. For many of these, we would like to change the data types for a few columns, using the `pd.to_numeric` and `pd.to_datetime` functions. We examine how to convert data types below.

### Converting Data Types to Numeric

The `object` type is a string. When we want to convert an object to numeric, we can use the `pd.to_numeric` function, as shown below:

In [ ]:
restaurants["SCORE"] = pd.to_numeric(restaurants["SCORE"])
restaurants["Latitude"] = pd.to_numeric(restaurants["Latitude"])
restaurants["Longitude"] = pd.to_numeric(restaurants["Longitude"])
restaurants.dtypes

###  Converting Data to Dates

Now let's convert the dates columns into the appropriate data types. Let's take a look at a few dates.

In [ ]:
restaurants["GRADE DATE"] = pd.to_datetime(restaurants["GRADE DATE"])
restaurants["RECORD DATE"] = pd.to_datetime(restaurants["RECORD DATE"])
restaurants["INSPECTION DATE"] = pd.to_datetime(restaurants["INSPECTION DATE"])

#### Note


In tricky cases, we may need to pass the `format` parameter, specifying the formatting of the date. For that, we need to understand first how to [parse dates using the Python conventions](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).


### Converting Data to Categorical Variables

This is less important, but sometimes we want to specify variables to be "Categorical". This is most commonly useful when we have variables that have an implicit order (e.g., the A/B/C grade of the restaurant).

In [ ]:
restaurants["BORO"] = pd.Categorical(restaurants["BORO"], ordered=False)
restaurants["GRADE"] = pd.Categorical(
    restaurants["GRADE"], categories=["A", "B", "C"], ordered=True
)
restaurants["VIOLATION CODE"] = pd.Categorical(
    restaurants["VIOLATION CODE"], ordered=False
)
restaurants["CRITICAL FLAG"] = pd.Categorical(
    restaurants["CRITICAL FLAG"], ordered=False
)
restaurants["ACTION"] = pd.Categorical(restaurants["ACTION"], ordered=False)
restaurants["CUISINE DESCRIPTION"] = pd.Categorical(
    restaurants["CUISINE DESCRIPTION"], ordered=False
)

restaurants["INSPECTION TYPE"] = pd.Categorical(
    restaurants["INSPECTION TYPE"], ordered=False
)

restaurants.dtypes

## Descriptive statistics



### Descriptive Statistics for Numeric Variables


#### Basic descriptive statistics for numeric variables

Given that SCORE is a numeric variable, we can get more detailed descriptive statistics for the variable using the `.describe()` command:

In [ ]:
restaurants["SCORE"].describe()

### Descriptive Statistics for Dates


In [ ]:
restaurants[["INSPECTION DATE", "GRADE DATE", "RECORD DATE"]].describe(datetime_is_numeric=True)

In [ ]:
restaurants["INSPECTION DATE"].describe(datetime_is_numeric=True)

In [ ]:
restaurants["GRADE DATE"].describe(datetime_is_numeric=True)

In [ ]:
restaurants["RECORD DATE"].describe(datetime_is_numeric=True)

### Descriptive Statistics for Categorical/string columns

We can also get quick statistics about the common values that appear in each column:

In [ ]:
restaurants["DBA"].value_counts()

In [ ]:
restaurants["CUISINE DESCRIPTION"].value_counts()

# Basic Data Manipulation Techniques

## Selecting a subset of the columns -- `filter()`

In a dataframe, we can specify the column(s) that we want to keep, and get back another dataframe with just the subset of the columns that we want to keep.
[`filter()` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html)

In [ ]:
restaurants

In [ ]:
restaurants.filter( 
    items = ["DBA", "GRADE", "GRADE DATE"] 
)

In [ ]:
columns = ["GRADE DATE", "VIOLATION CODE", "DBA", "SCORE"]

# Notice the use of "chain notation" below
# Chain notation means putting parentheses around
# the command and then having each operation in its
# own line
(
  restaurants
  .filter( items = columns )
  .head(10)
)


We can also use the `like` option to find all the column names that include a certain string. For example, to get all the columns that include the string `DATE`:

In [ ]:
restaurants.filter(
    like = 'DATE'
)

We can expand the functionality and also use regular expressions:

In [ ]:
restaurants.filter(
    regex = r'^C' # all the columns that start with C
)

## Renaming Columns -- `rename()`

To do the equivalent of `SELECT attr AS alias` in Pandas, we use the `rename` command, and pass a dictionary specifying which columns we want to rename:



In [ ]:
restaurants.rename(
    columns = {
      "CAMIS": "RESTID",
      "DBA": "REST_NAME",
      "BUILDING": "STREET_NUM",
      "BORO": "BOROUGH"
    }
)

## Selecting rows -- `query()`

To select rows, we can use the following approach, where we generate a list of boolean values, one for each row of the dataframe, and then we use the list to select which of the rows of the dataframe we want to keep"

[`query` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [ ]:
# Find all violations for restaurants with DBA being Starbucks
restaurants.query(' DBA == "STARBUCKS" ')

In [ ]:
# Find all violations with code 04L (i.e., "has mice")
# Notice the use of backquotes for attribute names that have space
restaurants.query(' `VIOLATION CODE` == "04L" ')

In [ ]:
# We can store the result in a dataframe called  has_mice
has_mice = restaurants.query(' `VIOLATION CODE` == "04L" ')
has_mice

In [ ]:
# The most frequent DBA names overall
restaurants["DBA"].value_counts()[:20]

In [ ]:
# List the most frequent DBA values in the dataframe
has_mice["DBA"].value_counts()[:20]

In [ ]:
has_mice["CAMIS"].value_counts()[:10]

In [ ]:
has_mice.query( ' CAMIS == "50015263" ' )

And we can use more complex conditions. 

In [ ]:
# AND in pandas is "&"
# OR in pandas is "|"

In [ ]:
has_mice_10012 = (
    restaurants
    .query(' `VIOLATION CODE` == "04L" & ZIPCODE == "10012" ')
    .filter( items = ['DBA', 'BUILDING', 'STREET', 'INSPECTION DATE'])
)

has_mice_10012

In [ ]:
has_mice_10012["DBA"].value_counts()[:30]

In [ ]:
has_mice_10012["DBA"].value_counts()[30::-1].plot(kind="barh")

## Selecting distinct values -- `drop_duplicates()`

We can do the equivalent of `SELECT DISTINCT` in Pandas by doing the following

In [ ]:
(
    has_mice_10012
    .filter( items = ['DBA', 'BUILDING', 'STREET'])
    .drop_duplicates()
)

## Sorting values -- `sort_values()`

And we can do the equivalent of `ORDER BY` by using the `.sort_values()

In [ ]:
(
    has_mice_10012
    .sort_values("INSPECTION DATE", ascending=False)
    .head(15)
)

In [ ]:
(
    has_mice_10012
    .sort_values(["INSPECTION DATE","DBA"], ascending=[False,True])
    .head(15)
)

## Defining New Columns -- `assign()` and `apply()`



### Using the `assign()` approach

The `assign` command applies a function to a dataframe and returns back a new dataframe with the new column(s).

In [ ]:
import numpy as np

# We define a function that will take as input a dataframe df
# and returns back a new column. This function computes
# the distance (in miles) from CityTech, given the lat/lon of the 
# other location
def distance(df):
  CityTech_lon = -73.9861
  CityTech_lat = 40.6973
  # The calculation below is simply the Pythagorean theorem.
  # The normalizing values are just for converting lat/lon differences
  # to miles
  distance = ((df.Latitude-CityTech_lat)/0.0146)**2 + ((df.Longitude-CityTech_lon)/0.0196)**2
  return np.sqrt(distance)

# This function combines STREET/BUILDING/BORO/ZIPCODE columns into one address
def combine_address(df):
  return (df.BUILDING + ' ' + df.STREET + ', '  + df.ZIPCODE).str.upper()

In [ ]:
# We now use the `assign` function to create two new columns
# using the logic in the functions above,
(
  restaurants
  .assign(
      distance_from_CityTech = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_CityTech'])
)

In [ ]:
# And let's eliminate now duplicates and sort by distance
(
  restaurants
  .assign(
      distance_from_CityTech = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_CityTech'])
  .query('distance_from_CityTech > 0') # eliminates NaN values from distance_from_CityTech
  .drop_duplicates()
  .sort_values('distance_from_CityTech')
  .head(20)
)

### Using the `apply` approach

The `apply` function allow the users to pass a function and apply it on every single row or column of a Pandas datarame. 

In [ ]:
!sudo pip3 install -q -U geopy

from geopy import distance

# A bit more accurate distance calculation, which returns back
# the distance in miles. However, we cannot pass a dataframe
# to the function but only individual values
def distance_from_CityTech_geodesic(row):
  CityTech_lon = -73.9861
  CityTech_lat = 40.6973
  CityTech = (CityTech_lat, CityTech_lon)
  rest = (row.Latitude, row.Longitude)
  #if pd.isnull(row.Latitude) or pd.isnull(row.Longitude):
  #  return None
  return distance.distance(CityTech, rest).miles


In [ ]:
# We now create a smaller version of the dataset with just
# the names/address/lon/lat of the restaurants
rest_names_locations = (
    restaurants
    .assign(
      address = combine_address
    )
    .filter(items = ['CAMIS','DBA','address','Longitude', 'Latitude'])
    .query(' Longitude==Longitude ') # idiomatic expression for saying IS NOT NULL
    .query(' Latitude==Latitude ') # idiomatic expression for saying IS NOT NULL
    .drop_duplicates()
)

rest_names_locations

In [ ]:
# We will now apply the function distance_from_CityTech_geodesic 
# to every row of the dataset:
rest_names_locations.apply(distance_from_CityTech_geodesic, axis='columns')


In [ ]:
# We will now save the result into a new column
rest_names_locations['distance_from_CityTech']=rest_names_locations.apply(distance_from_CityTech_geodesic, axis='columns')

In [ ]:
# Let's see how many restaurants are within half a mile from NYU :)
(
    rest_names_locations
    .query('distance_from_CityTech < 0.5')
    .sort_values('distance_from_CityTech')
)

## Aggregation Function -- `agg()`

In [ ]:
restaurants['SCORE'].agg('mean')

In [ ]:
restaurants['SCORE'].agg(['mean','std','count','nunique'])

In [ ]:
restaurants.agg(
    {
        'SCORE': ['mean','std','count','nunique'],
        'CAMIS':  ['nunique','count']
     }
    )

In [ ]:
restaurants.agg(
        num_scored_violations = ('SCORE', 'count'),
        mean_score = ('SCORE', 'mean'),
        std_score  = ('SCORE', 'std'),
        num_entries = ('CAMIS',  'count'),
        num_restaurants = ('CAMIS',  'nunique'),
  )

## Calculating aggegates per groups -- `groupby()`

In [ ]:
restaurants.groupby('GRADE DATE').agg({'SCORE': 'mean'})

In [ ]:
(
  restaurants
  .groupby('GRADE DATE')
  .agg(
      score_mean = ('SCORE', 'mean'), 
      graded_restaurants = ('CAMIS', 'nunique')
    )
  .tail(500)
  .head(20)
)

## Pivot Tables

[Pivot tables](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) is one of the most commonly used exploratory tools, and in Pandas they are extremely flexible. 

For example, let's try to count the number of restaurants that are inspected every day. 

In [ ]:
# Count the number of CAMIS values that appear on each date

pivot = pd.pivot_table(
    data=restaurants,
    index="GRADE DATE",  # specifies the rows
    values="CAMIS",  # specifies the content of the cells
    aggfunc="count",  # we ask to count how many different CAMIS values we see
)

In [ ]:
pivot

#### Changing date granularity 

We can also use the [resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) command to change the frequency from one day, to, say, 7 days. Then we can compute, say, the average (`mean()`) for these days, or the total number (`sum()`) of inspections.

In [ ]:
pivot.resample("1W").sum().tail(100)

#### Pivot Table with two (or more) variables)

We would like to break down the results by borough, so we add the `column` parameter.

In [ ]:
pivot2 = pd.pivot_table(
    data=restaurants,  #
    index="INSPECTION DATE",
    columns="BORO",
    values="CAMIS",
    aggfunc="count",
)
pivot2.head(10)

##### Deleting rows and columns

Now, you will notice that there are a few columns and rows that are just noise. The first row with date *'1900-01-01'* is clearly noise, and the *'0'* column is also noise. We can use the `drop` command of Pandas to drop these.

In [ ]:
# The axis='index' (or axis=0) means that we delete a row with that index value
pivot2 = pivot2.drop(pd.to_datetime("1900-01-01"), axis="index")

In [ ]:
# The axis='columns' (or axis=1) means that we delete a columns with that value
pivot2 = pivot2.drop("0", axis="columns")

In [ ]:
pivot2.tail(5)

## (Optional, FYI) Advanced Pivot Tables

We can also add multiple attributes in the index and columns. It is also possible to have multiple aggregation functions, and we can even define our own aggregation functions.

In [ ]:
# We write a function that returns the
# number of unique items in a list x
def count_unique(x):
    return len(set(x))


# We break down by BORO and GRADE, and also calculate
# inspections in unique (unique restaurants)
# and non-unique entries (effectuvely, violations)
pivot_advanced = pd.pivot_table(
    data=restaurants,  #
    index="GRADE DATE",
    columns=["BORO", "GRADE"],
    values="CAMIS",
    aggfunc=["count", count_unique],
)

# Take the total number of inspections (unique and non-unique)
agg = pivot_advanced.resample("1M").sum()

# Show the last 5 entries and show the transpose (.T)
agg.tail().T

### Exercise 1 

Now let's do the same exercise, but instead of counting the number of inspections, we want to compute the average score assigned by the inspectors. Hint: We will need to change the `values` and the `aggfunc` parameters in the `pivot_table` function above.

In [ ]:
# your code here

#### Solution

In [ ]:
pivot = pd.pivot_table(
    data=restaurants,
    index="INSPECTION DATE",  # specifies the rows
    values="SCORE",  # specifies the content of the cells
    aggfunc="mean",  # compute the average SCORE
)